In machine learning, the train-dev-test split is a common technique used to evaluate the performance of a model and prevent overfitting. The dataset is divided into three subsets: the training set, the development (or validation) set, and the test set.

The benchmark comprises 8628 sentence pairs. This is the breakdown according to genres and train-dev-test splits:

                train  dev test total 
        -----------------------------
        news     3299  500  500  4299
        caption  2000  625  525  3250
        forum     450  375  254  1079
        -----------------------------
        total    5749 1500 1379  8628

- [x] Calculate average document length
- [x] Calculate average vocabulary size
- [ ] Calculate average sentence length
- [ ] Check if scores are balanced
- [ ] Topic clustering
- [ ] Keyword search
- [ ] Word2Vec embedding

In [8]:
import pandas as pd

def on_bad_line(values):
    return values[:7]

columns_mapping = {
    0: 'genre',
    1: 'filename',
    2: 'year',
    3: 'index',
    4: 'score',
    5: 'sentence1',
    6: 'sentence2'
}

# quotings 3 = csv.QUOTE_NONE
df = pd.read_csv('Dataset/sts-train.csv', sep="\t", on_bad_lines=on_bad_line, engine='python', header=None, encoding='utf-8', quoting=3).rename(columns=columns_mapping)
print(f'shape of the Dataframe {df.shape}')
df.head(10)

shape of the Dataframe (5749, 7)


,genre,filename,year,index,score,sentence1,sentence2
0,main-captions,MSRvid,2012test,1,5.00,A plane is taking off.,An air plane is taking off.
1,main-captions,MSRvid,2012test,4,3.80,A man is playing a large flute.,A man is playing a flute.
2,main-captions,MSRvid,2012test,5,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,main-captions,MSRvid,2012test,6,2.60,Three men are playing chess.,Two men are playing chess.
4,main-captions,MSRvid,2012test,9,4.25,A man is playing the cello.,A man seated is playing the cello.
5,main-captions,MSRvid,2012test,11,4.25,Some men are fighting.,Two men are fighting.
6,main-captions,MSRvid,2012test,12,0.50,A man is smoking.,A man is skating.
7,main-captions,MSRvid,2012test,13,1.60,The man is playing the piano.,The man is playing the guitar.
8,main-captions,MSRvid,2012test,14,2.20,A man is playing on a guitar and singing.,A woman is playing an acoustic guitar and sing...
9,main-captions,MSRvid,2012test,16,5.00,A person is throwing a cat on to the ceiling.,A person throws a cat on the ceiling.


In [12]:
import numpy as np
import nltk
nltk.download("punkt")
from nltk.tokenize import word_tokenize
from string import punctuation
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\utente\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
sentences1 = df["sentence1"]
sentences2 = df["sentence2"]
sentences = sentences1 + " " + sentences2

all_sentences = np.ravel(sentences.values)

A plane is taking off. An air plane is taking off.
["''", "'accept", "'activities", "'alone", "'amazing", "'an", "'and", "'anti-terror", "'apostasy", "'are", "'attractive", "'azawad", "'ban", "'beaten", "'bestest", "'betrayed", "'biggest", "'blade", "'blasphemy", "'bled", "'bookkeeper", "'buried", "'ca", "'capture", "'cares", "'catches", "'cautiously", "'compares", "'concern", "'concerns", "'conclusively", "'concrete", "'constructive", "'costing", "'could", "'crashes", "'credibility", "'d", "'determined", "'did", "'die", "'dozens", "'exploits", "'exploratory", "'failed", "'faked", "'final", "'fires", "'flytilla", "'for", "'framework", "'freezes", "'friday", "'friends", "'gang-raped", "'gas", "'given", "'glee", "'grave", "'grenade", "'groundless", "'has", "'hearing", "'heavy", "'hit", "'honor", "'hunger", "'improved", "'in", "'incorrect", "'india", "'insider", "'inspiration", "'intensive", "'iran", "'iron", "'israeli", "'it", "'jihad", "'jihadi", "'judge", "'justice", "'kill", "'killed"

In [ ]:
regex = '[' + punctuation + ']'
print(regex)

In [ ]:
clean_list = [re.sub(regex,'',string) for string in all_sentences]
print(clean_list)

In [ ]:
words_with_duplicates = [word.lower() for string in clean_list for word in word_tokenize(string)]
words = list(set(words_with_duplicates))

sorted_words = sorted(words)

print(sorted_words)

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\utente\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
from nltk.corpus import stopwords

In [23]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [32]:
words_nostopwords = [w for w in words_with_duplicates if w not in stopwords.words('english')]
counts_nostopwords = nltk.Counter(words_nostopwords)
counts_nostopwords.most_common(20)

[('man', 1092),
 ('woman', 673),
 ('two', 491),
 ('said', 468),
 ('dog', 445),
 ('us', 424),
 ('playing', 400),
 ('white', 334),
 ('black', 307),
 ('killed', 291),
 ('percent', 263),
 ('new', 245),
 ('people', 234),
 ('syria', 220),
 ('china', 203),
 ('running', 183),
 ('police', 176),
 ('president', 176),
 ('water', 172),
 ('person', 169)]

In [33]:
vocabulary = sorted(list(set(words_nostopwords)))
print(len(vocabulary))
print(vocabulary)

12253
['0', '004', '005', '006', '008', '01', '011', '012', '013pc', '015', '018', '019', '02', '0204', '024', '0241', '026', '026pc', '027', '028', '029', '03', '030', '031', '032', '036', '037', '039', '04', '041', '044', '045', '046', '047', '048', '049', '05', '0500', '051', '052', '056', '06', '062', '064', '066', '067', '07', '075pc', '08', '081', '083', '0841', '086', '088pc', '09', '0932', '094', '098', '1', '10', '100', '1000', '10000', '100000', '1004', '1006', '100th', '101', '101569', '103', '104', '104679', '104th', '105', '1055', '10557', '10578', '106', '1076', '108', '10852', '1089', '1093', '10b', '10city', '10month', '10monthold', '10year', '11', '110', '11000', '11132', '112', '113', '1130am', '1132', '114', '115', '1156', '11597', '116', '11724', '11725', '1174', '11741', '11784', '11785', '117kg', '118', '118350', '11857', '11860', '11884', '119', '1191', '11914', '11935', '11am', '11month', '11th', '12', '120', '1201', '120650', '1214', '1219', '122', '124', '125'